# 3. KL Divergence, Fisher Information, and the Natural Gradient


As seen before, the Kullback-Leibler divergence between two distributions is an asymmetric measure of how different two distributions are. Consider two distributions over the same space given by densities $p(x)$ and $q(x)$. The KL divergence between two continuous distributions, $q$ and $p$ is defined as,


\begin{align*}
D_{KL}(p\|q) 
& = \int_{-\infty}^\infty p(x)\log\frac{p(x)}{q(x)}dx\\
& = \int_{-\infty}^\infty p(x)\log p(x)dx - \int_{-\infty}^\infty p(x)\log q(x)dx\\
& = \mathbb{E}_{x\sim p(x)}[\log p(x)] - \mathbb{E}_{x\sim p(x)}[\log(q(x)].
\end{align*}

A nice property of KL divergence is that it invariant to parametrization. This means, KL divergence evaluates to the same value no matter how we parametrize the distributions $P$ and $Q$. For e.g, if $P$ and $Q$ are in the exponential family, the KL divergence between them is the same whether we are using natural parameters, or canonical parameters, or any arbitrary reparametrization.

Now we consider the problem of fitting model parameters using gradient descent (or stochastic gradient descent). As seen previously, fitting model parameters using Maximum Likelihood is equivalent to minimizing the KL divergence between the data and the model. While KL divergence is invariant to parametrization, the gradient w.r.t the model parameters (i.e, direction of steepest descent) is not invariant to parametrization. To see its implication, suppose we are at a particular value of parameters (either randomly initialized, or mid-way through the optimization process). The value of the parameters correspond to some probability distribution (and in case of regression, a conditional probability distribution). If we follow the direction of steepest descent from the current parameter, take a small step along that direction to a new parameter, we end up with a new distribution corresponding to the new parameters. The non- invariance to reparametrization means, a step of fixed size in the parameter space could end up in a distribution that could either be extremely far away in $D_{KL}$ from the previous distribution, or on the other hand not move very much at all w.r.t $D_{KL}$ from the previous distributions.

This is where the natural gradient comes into picture. It is best introduced in contrast with the usual gradient descent. In the usual gradient descent, we first choose the direction by calculating the gradient of the MLE objective w.r.t the parameters, and then move a magnitude of step size (where size is measured in the parameter space) along that direction. Whereas in natural gradi- ent, we first choose a divergence amount by which we would like to move, in the $D_{KL}$ sense. This effectively gives us a perimeter around the current parameters (of some arbitrary shape), such that points along this perimeter correspond to distributions which are at an equal $D_{KL}$-distance away from the current parameter. Among the set of all distributions along this perimeter, we move to the distribution that maximizes the objective (i.e minimize $D_{KL}$ between data and itself) the most. This approach makes the optimization process invariant to parametrization. That means, even if we chose a new arbitrary reparametrization, by starting from a particular distribution, we always descend down the same sequence of distributions towards the optimum.

In the rest of this problem, we will construct and derive the natural gradient update rule. For that, we will break down the process into smaller sub-problems, and give you hints to answer them. Along the way, we will encounter important statistical concepts such as the score function and Fisher Information (which play a prominant role in Statistical Learning Theory as well). Finally, we will see how this new natural gradient based optimization is actually equivalent to Newton’s method for Generalized Linear Models.

Let the distribution of a random variable $Y$ parameterized by $\theta\in\mathbb{R}^n$ be $p(y;\theta)$.

## (a) [3 points] Score function


The score function associated with $p(y;\theta)$ is defined as $\nabla_\theta \log p(y;\theta)$, which signifies the sensitivity of the likelihood function with respect to the parameters. Note that the score
function is actually a vector since it’s the gradient of a scalar quantity with respect to the
vector $\theta$.

Recall that $\mathbb{E}_{x\sim p(y)}[g(y)] =\int_{-\infty}^\infty g(y)p(y)dy $. Using this fact, show that the expected value of the score is $)$, i.e.

\begin{align*}
\mathbb{E}_{x\sim p(y;\theta)}\left[\nabla_{\theta'} \log p(y;\theta')|_{\theta'=\theta}\right] = 0.
\end{align*}

### Asnwer:

\begin{align*}
\mathbb{E}_{x\sim p(y;\theta)}\left[\nabla_{\theta'} \log p(y;\theta')|_{\theta'=\theta}\right]
& = \mathbb{E}_{x\sim p(y;\theta)}\left[\frac{1}{p(y;\theta)}\frac{\partial p(y;\theta)}{\partial \theta}\right]\\
& = \int_{-\infty}^\infty p(y;\theta) \frac{1}{p(y;\theta)}\frac{\partial p(y;\theta)}{\partial \theta} dy\\
& = \int_{-\infty}^\infty \frac{\partial p(y;\theta)}{\partial \theta} dy\\
& = \frac{\partial}{\partial \theta}\int_{-\infty}^\infty  p(y;\theta) dy\\
& = \frac{\partial}{\partial \theta} 1 \\
& = 0.
\end{align*}

## (b) [2 points] Fisher Information

Let us now introduce a quantity known as the Fisher information. It is defined as the covariance matrix of the score function,

\begin{align*}
\mathcal{I}(\theta) = 
{\rm Cov}_{x\sim p(y;\theta)}\left[\nabla_{\theta'} \log p(y;\theta')|_{\theta'=\theta}\right].
\end{align*}

Intuitively, the Fisher information represents the amount of information that a random variable $Y$ carries about a parameter $\theta$ of interest. When the parameter of interest is a vector (as in our case, since $\theta\in\mathbb{R}^n$), this information becomes a matrix. Show that the Fisher information can equivalently be given by

\begin{align*}
\mathcal{I}(\theta) =
\mathbb{E}_{x\sim p(y;\theta)}\left[\nabla_{\theta'} \log p(y;\theta')\nabla_{\theta'} \log p(y;\theta')^T|_{\theta'=\theta}\right].
\end{align*}

Note that the Fisher Information is a function of the parameter. The parameter of the Fisher information is both a) the parameter value at which the score function is evaluated, and b) the parameter of the distribution with respect to which the expectation and variance is calculated.

### Answer:

We know that
\begin{align*}
{\rm Cov}(X) = \mathbb{E}\left[\left(X-\mathbb{E}(X)\right)\left(X-\mathbb{E}(X)\right)^T\right]
\end{align*}
and, hence, when $\mathbb{E}(X) = 0$
\begin{align*}
{\rm Cov}(X) = \mathbb{E}\left[XX^T\right].
\end{align*}
Consequently, 

\begin{align*}
\mathcal{I}(\theta) 
& = {\rm Cov}_{x\sim p(y;\theta)}\left[\nabla_{\theta'} \log p(y;\theta')|_{\theta'=\theta}\right] \\
& = \mathbb{E}_{x\sim p(y;\theta)}\left[\nabla_{\theta'} \log p(y;\theta')\nabla_{\theta'} \log p(y;\theta')^T|_{\theta'=\theta}\right].
\end{align*}



## (c) [5 points] Fisher Information (alternate form)

It turns out that the Fisher Information can not only be defined as the covariance of the
score function, but in most situations it can also be represented as the expected negative Hessian of the log-likelihood.

Show that
\begin{align*}
\mathcal{I}(\theta) = \mathbb{E}_{x\sim p(y;\theta)}\left[-\nabla^2_{\theta'} \log p(y;\theta')|_{\theta'=\theta}\right].
\end{align*}

**Remark.** The Hessian represents the curvature of a function at a point. This shows that the expected curvature of the log-likelihood function is also equal to the Fisher information matrix. If the curvature of the log-likelihood at a parameter is very steep (i.e, Fisher Information is very high), this generally means you need fewer number of data samples to a estimate that parameter well (assuming data was generated from the distribution with those parameters), and vice versa. The Fisher information matrix associated with a statistical model parameterized by $\theta$ is extremely important in determining how a model behaves as a function of the number of training set examples.

### Answer:

Note that 
\begin{align*}
\nabla_{\theta'} \log p(y;\theta') 
& = \frac{1}{p(y;\theta')}\nabla_{\theta'} p(y;\theta')
\end{align*}

and 

\begin{align*}
\nabla^2_{\theta'} \log p(y;\theta') 
& = \frac{\partial}{\partial \theta'}\Big(\frac{1}{p(y;\theta')}\nabla_{\theta'} p(y;\theta')\Big)\\
& = \frac{1}{p(y;\theta')^2}\nabla_{\theta'} p(y;\theta')\nabla_{\theta'} p(y;\theta')^T + \frac{1}{p(y;\theta')}\nabla^2_{\theta'} p(y;\theta').
\end{align*}

Therefore, 

\begin{align*}
\mathbb{E}_{x\sim p(y;\theta)}\left[\nabla^2_{\theta'} \log p(y;\theta')|_{\theta'=\theta}\right]
& = \mathbb{E}_{x\sim p(y;\theta')}\left[-\frac{1}{p(y;\theta')^2}\nabla_{\theta'} p(y;\theta')\nabla_{\theta'} p(y;\theta')^T + \frac{1}{p(y;\theta')}\nabla^2_{\theta'} p(y;\theta')|_{\theta =\theta'}\right]\\
& = -\mathbb{E}_{x\sim p(y;\theta)}\left[\nabla_{\theta'} \log p(y;\theta')\nabla_{\theta'} \log p(y;\theta')^T|_{\theta'=\theta}\right] +
\mathbb{E}_{x\sim p(y;\theta)}\left[\frac{1}{p(y;\theta')}\nabla^2_{\theta'} p(y;\theta')|_{\theta =\theta'}\right]\\
& = -\mathcal{I}(\theta) + 0
\end{align*}

For the last equality, we used the follwoing fact.

\begin{align*}
\mathbb{E}_{x\sim p(y;\theta)}\left[\frac{1}{p(y;\theta')}\nabla^2_{\theta'} p(y;\theta')|_{\theta =\theta'}\right]
& = \int_{-\infty}^{\infty} p(y;\theta)\frac{1}{p(y;\theta)}\nabla^2_{\theta} p(y;\theta)dy\\
& = \int_{-\infty}^{\infty} \nabla^2_{\theta} p(y;\theta)dy\\
& = \nabla^2_{\theta}\int_{-\infty}^{\infty} p(y;\theta)dy\\
& = \nabla^2_{\theta} 1 \\
& = 0.
\end{align*}

## (d) [5 points] Approximating $D_{KL}$ with Fisher Information

As we explained at the start of this problem, we are interested in the set of all distributions that are at a small fixed $D_{KL}$ distance away from the current distribution. In order to calculate DKL between $p(y;\theta)$ and $p(y;\theta+d)$, where $d\in\mathbb{R}^n$ is a small magnitude “delta” vector, we approximate it using the Fisher Information at $\theta$. Eventually $d$ will be the natural gradient update we will add to $\theta$. To approximate the KL-divergence with FisherInfomration, we will start with the Taylor Series expansion of $D_{KL}$ and see that the Fisher Information pops up in the expansion.

Show that $D_{KL}(p_{\theta}\|p_{\theta+d})\approx \frac{1}{2}d^T\mathcal{I}(\theta)d.$

**Hint:** Start with the Taylor Series expansion of $D_{KL}(p_{\theta}\|p_{\tilde{\theta}})$ where $\theta$ is a constant and $\tilde{\theta}$ is a variable. Later set $\tilde{\theta} = \theta+d$. Recall that the Taylor Series allows us to approximate a scalar function $f(\tilde{\theta})$ near $\theta$ by:

\begin{align*}
f(\tilde{\theta}) \approx f(\theta) + (\tilde{\theta} - \theta)^T\nabla_{\theta'}f(\theta')|_{\theta' = \theta}
+ \frac{1}{2}(\tilde{\theta} - \theta)^T\big(\nabla^2_{\theta'}f(\theta')|_{\theta' = \theta}\big)(\tilde{\theta} - \theta).
\end{align*}

### Answer:

Set 
\begin{align*}
f(\tilde{\theta}) &= \mathbb{E}_{y\sim p_{\theta}(y)}\left[D_{KL}(p_{\theta}(y)\|p_{\tilde{\theta}}(y))\right]\\
& = \mathbb{E}_{y\sim p_{\theta}(y)}\left[\log p_{\theta}(y)\right] - \mathbb{E}_{y\sim p_{\theta}(y)}\left[\log p_{\tilde{\theta}}(y))\right].
\end{align*}

First, note that $f(\theta) = 0$.
Using Parts (a) and (c), we have 

\begin{align*}
\nabla_{\tilde{\theta}} f(\tilde{\theta})|_{\tilde{\theta}=\theta} 
& = - \mathbb{E}_{y\sim p_{\theta}(y)}\left[\nabla_{\tilde{\theta}}\log p_{\tilde{\theta}}(y)|_{\tilde{\theta}=\theta}\right]\\
& = 0
\end{align*}

and 

\begin{align*}
\nabla^2_{\tilde{\theta}} f(\tilde{\theta})|_{\tilde{\theta}=\theta} 
& = - \mathbb{E}_{y\sim p_{\theta}(y)}\left[\nabla^2_{\tilde{\theta}}\log p_{\tilde{\theta}}(y)|_{\tilde{\theta}=\theta}\right]\\
& = \mathcal{I}(\theta).
\end{align*}

Remind that $\tilde{\theta} = \theta +d$. Using the Taylor Series to approximate the scalar function $f(\tilde{\theta})$ near $\theta$, we have
\begin{align*}
f(\tilde{\theta}) 
& \approx f(\theta) + (\tilde{\theta} - \theta)^T\nabla_{\theta'}f(\theta')|_{\theta' = \theta}
+ \frac{1}{2}(\tilde{\theta} - \theta)^T\big(\nabla^2_{\theta'}f(\theta')|_{\theta' = \theta}\big)(\tilde{\theta} - \theta)\\
 & = \frac{1}{2}d^T\mathcal{I}(\theta)d.
\end{align*}


## (e) [8 points] Natural Gradient

Now we move on to calculating the natural gradient. Recall that we want to maximize the log-likelihood by moving only by a fixed $D_{KL}$ distance from the current position. In the previous sub-question we came up with a way to approximate DKL distance with Fisher Information. Now we will set up the constrained optimization problem that will yield the natural gradient update d. Let the log-likelihood objective be $\ell(\theta)= \log p(y,\theta)$. Let the $D_{KL}$ distance we want to move by, be some small positive constant $c$. The natural gradient update $d^*$ is

\begin{equation*}
d^* = \arg\max_{d}\ell(\theta+d) \quad  \, subject\, to\,\quad D_{KL}(p_\theta\|p_{\theta+d}) = c.
\label{1} \tag{1}
\end{equation*}

First we note that we can use Taylor approximation on $\ell(\theta+d)\approx \ell(\theta) +d^T\nabla_{\theta'}\ell(\theta')|_{\theta' =\theta}$. Also note that we calculated the Taylor approximation $D_{KL}(p_{\theta}\|p_{\theta+d})$ in the previous sub-problem. We shall substitute both these approximations into the above constrainted optimization problem.

In order to solve this constrained optimization problem, we employ the method of Lagrange multipliers. If you are familiar with Lagrange multipliers, you can proceed directly to solve for $d^*$. If you are not familiar with Lagrange multipliers, here is a simplified introduction. (You may also refer to a slightly more comprehensive introduction in the Convex Optimization section notes, but for the purposes of this problem, the simplified introduction provided here should suffice).

Consider the following constrained optimization problem
\begin{align*}
d^* = \arg\max_{d}f(d) \quad  \, subject\, to\,\quad g(d) = c.
\end{align*}
The function $f$ is the objective function and $g$ is the constraint. We instead optimize the Lagrangian $\mathcal{L}(d,\lambda)$, which is defined as

\begin{align*}
\mathcal{L}(d,\lambda) = f(d)- \lambda[g(d)-c].
\end{align*}
with respect to both d and $\lambda$. Here $\lambda\in \mathbb{R}_+$ is called the Lagrange multiplier. In order to
optimize the above, we construct the following system of equations:

\begin{equation*}
\nabla_d\mathcal{L}(d,\lambda)=0,
\label{a} \tag{a}
\end{equation*}

\begin{equation*}
\nabla_\lambda\mathcal{L}(d,\lambda)=0
\label{b} \tag{b}
\end{equation*}

So we have two equations (a and b above) with two unknowns ($d$ and $\lambda$), which can be sometimes be solved analytically (in our case, we can).

The following steps guide you through solving the constrained optimization problem:

- Construct the Lagrangian for the constrained optimization problem (1) with the Taylor approximations substituted in for both the objective and the constraint.


- Then construct the system of linear equations (like (a) and (b)) from the Lagrangian you obtained.


- From (a), come up with an expression for $d$ that involves $\lambda$. At this stage we have already found the "direction" of the natural gradient d, since $\lambda$ is only a positive scaling constant. For most practical purposes, the solution we obtain here is sufficient. This is because we almost always include a learning rate hyperparameter in our optimization algorithms, or perform some kind of a line search for algorithmic stability. This can make the exact calculation of $\lambda$ less critical. Let's call this expression $\tilde{d}$ (involving $\lambda$) as the unscaled natural gradient. Clearly state what is d ̃ as a function of $\lambda$. 
The remaining steps are to figure out the value of the scaling constant $\lambda$ along the direction of $d$, for completeness.


- Plugin that expression for $d$ into (b). Now we have an equation that has $\lambda$ but not $d$. Come up with an expression for $\lambda$ that does not include $d$.


- Plug that expression for $\lambda$ (without $d$) back into (a). Now we have an equation that has $d$ but not $\lambda$. Come up with an expression for $d$ that does not include $\lambda$.

The expression for $d$ obtained this way will be the desired natural gradient update $d^*$. Clearly state and highlight your final expression for $d^*$. This expression cannot include $\lambda$.

### Answer:

Remind that $\ell(\theta)= \log p(y,\theta)$ and hence 
\begin{align*}
\nabla_{\theta'}\ell(\theta')|_{\theta' =\theta} = \frac{1}{p(y,\theta)}\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}.
\end{align*} 
Using Taylor approximation, we can rewrite (1) as follows:
\begin{equation*}
d^* = \arg\max_{d}  \ell(\theta) +d^T\nabla_{\theta'}\ell(\theta')|_{\theta' =\theta} \quad  \, subject\, to\,\quad \frac{1}{2}d^T\mathcal{I}(\theta)d = c.
\label{a} \tag{a}
\end{equation*}

Since $\ell(\theta)$ and $\frac{1}{p(y,\theta)}$ are positive constants w.r.t. $d$, this could be simplified even more to  

\begin{equation*}
d^* = \arg\max_{d} d^T\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta} \quad  \, subject\, to\,\quad \frac{1}{2}d^T\mathcal{I}(\theta)d = c.
\label{b} \tag{b}.
\end{equation*}

The Lagrangian $\mathcal{L}(d,\lambda)$ for the constrained optimization problem (b) is 

\begin{align*}
\mathcal{L}(d,\lambda)& = d^T\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta} - \lambda(\frac{1}{2}d^T\mathcal{I}(\theta)d - c).
\end{align*}


We can insted optimize $\mathcal{L}(d,\lambda)$ w.r.t both $d$ and $\lambda$, where $\lambda\in\mathbb{R}_+$. To optimize $\mathcal{L}(d,\lambda)$, we construct the following system of equations:

\begin{equation*}
\nabla_d\mathcal{L}(d,\lambda) = \nabla_{\theta'}p(y,\theta)|_{\theta'=\theta} - \lambda\mathcal{I}(\theta)d = 0,
\label{i} \tag{i}
\end{equation*}

\begin{equation*}
\nabla_\lambda\mathcal{L}(d,\lambda) = -\frac{1}{2}d^T\mathcal{I}(\theta)d + c = 0
\label{ii} \tag{ii}
\end{equation*}

From (i), we obtain the unscaled natural gradient $\tilde{d}(\lambda)$ as follwos,   

\begin{equation*}
\tilde{d} = \frac{1}{\lambda}\mathcal{I}(\theta)^{-1}\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}.
\end{equation*}

Pluging this expression into (ii), we have 

\begin{align*}
\frac{1}{2\lambda^2}\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)^T\mathcal{I}(\theta)^{-1}\mathcal{I}(\theta)
\mathcal{I}(\theta)^{-1}\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)^T
& = \frac{1}{2\lambda^2}\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)^T
\mathcal{I}(\theta)^{-1}\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)\\
& = c.
\end{align*}

Therefore,

\begin{align*}
\lambda = \sqrt{\frac{1}{2c}\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)^T
\mathcal{I}(\theta)^{-1}\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)}.
\end{align*}

Plug it into the formula of $\tilde{d}$, we obtain 

\begin{align*}
d^* =  \sqrt{\frac{2c}{\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)^T
\mathcal{I}(\theta)^{-1}\left(\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}\right)}}\mathcal{I}(\theta)^{-1}\nabla_{\theta'}p(y,\theta)|_{\theta'=\theta}.
\end{align*}


## (f) [2 points] Relation to Newton’s Method

After going through all these steps to calculate the natural gradient, you might wonder if this is something used in practice. We will now see that the familiar Newton's method that we studied earlier, when applied to Generalized Linear Models, is equivalent to natural gradient on Generalized Linear Models. While the two methods (Netwon's and natural gradient) agree on GLMs, in general they need not be equivalent.

Show that the direction of update of Newton's method, and the direction of natural gradient, are exactly the same for Generalized Linear Models. You may want to recall and cite the results you derived in problem set 1 question 4 (Convexity of GLMs). For the natural gradient, it is sufficient to use $\tilde{d}$, the unscaled natural gradient.

### Answer:


In view of Part (e), to maximize $\ell(\theta)=\log p(y;\theta)$ (MLE), the update rule based on natural gradient method is 

\begin{equation*}
\theta_{new} := \theta_{old} + \alpha\mathcal{I}(\theta_{old})^{-1}\nabla_{\theta}p(y,\theta).
\label{i} \tag{i}
\end{equation*}

To recap, an exponential family distribution is one whose probability density can be represented
$$p(y,\eta)=b(y)\exp\left(\eta^TT(y) - a(\eta)\right)$$
where $\eta$ is the natural parameter of the distribution. Moreover, in a Generalized Linear Model, $\eta$ is modeled as $\theta^Tx$, where $x\in \mathbb{R}^n$ is the input features of the example, and $\theta\in\mathbb{R}^n$ is learnable parameters. 


In view of problem set 1 question 4 (Convexity of GLMs) Part (c), the second derivative (i.e., Hessian) of the function $\ell(\theta) = \log p(y;\theta)$, $H(\theta)$, w.r.t the model parameter $\theta$ is a constant function w.r.t. $y$ which implies   

\begin{align*}
\mathcal{I}(\theta) 
& = \mathbb{E}_{x\sim p(y;\theta)}\left[-\nabla^2_{\theta'} \log p(y;\theta')|_{\theta'=\theta}\right]\\
& = \mathbb{E}_{x\sim p(y;\theta)}\left[-H(\theta)\right]\\
& = -H(\theta).
\end{align*}

Plug it into Equation (i), the natural gradient update rule for Generalized Linear Model would be 

\begin{align*}
\theta_{new} := \theta_{old} - \alpha H(\theta_{old})^{-1}\nabla_{\theta_{old}}p(y,\theta_{old}).
\end{align*}

Newton's method suggests the follwoing update rule to maximize $l(\theta)$:

\begin{align*}
\theta_{new} := \theta_{old} - H(\theta_{old})^{-1}\nabla_{\theta_{old}}p(y,\theta_{old}).
\end{align*}

Note that $l(\theta)$ is a negative semi-definite function. Therefore,  Newton's method convereges to its global maximum. 

Comparing these two latter update rules, it is clear that the direction of update of Newton's method and the direction of natural gradient are exactly the same.